Importing libraries

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential,regularizers
from tensorflow.keras.layers import Dense,Dropout,BatchNormalization,Conv2D,MaxPool2D,Flatten
from tensorflow.keras.optimizers import Adam
import warnings
warnings.filterwarnings('ignore')

Loading CSV data

In [3]:
df=pd.read_csv(r'C:\Users\DELL\Downloads\archive\train.csv')
df.shape

(3662, 2)

In [31]:
df.head()

,id_code,diagnosis
0,000c1434d8d7,2
1,001639a390f0,4
2,0024cdab0c1e,1
3,002c21358ce6,0
4,005b95c28852,0


Data Preprocessing

In [4]:
## Apply extension .png to image ids
def apply_extention(x):
  return x+".png"

df['id_code']=df['id_code'].apply(apply_extention)

In [5]:
## Define stages of db retinopathy in label column
diagnosis_dict={
  0:'No_DR',
  1:'Mild',
  2:'Moderate',
  3:'Severe',
  4:'Proliferate_DR'
}

df['stage']=df['diagnosis'].map(diagnosis_dict)
df.head()

,id_code,diagnosis,stage
0,000c1434d8d7.png,2,Moderate
1,001639a390f0.png,4,Proliferate_DR
2,0024cdab0c1e.png,1,Mild
3,002c21358ce6.png,0,No_DR
4,005b95c28852.png,0,No_DR


In [6]:
## Splitting data into training and testing data
from sklearn.model_selection import train_test_split
train,test=train_test_split(df,test_size=0.2,random_state=42)

Setting up Image data generator

In [7]:
## Image data generator
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1 / 255.0,
                                                               rotation_range = 10,
                                                               zoom_range = 0.30,
                                                               shear_range = 0.30,
                                                               fill_mode = "nearest"
                                                               )
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1 / 255.0)

In [8]:
## Create and copy images to working directories
train_generator = train_datagen.flow_from_dataframe(dataframe = train,
                                                    directory = r'C:\Users\DELL\Downloads\archive\gaussian_filtered_images\gaussian_filtered_images',
                                                    x_col = 'id_code',
                                                    y_col = 'stage',
                                                    target_size=(224, 224),
                                                    color_mode='rgb',
                                                    class_mode='categorical',
                                                    batch_size=16)
test_generator = test_datagen.flow_from_dataframe(dataframe = test,
                                                  directory = r'C:\Users\DELL\Downloads\archive\gaussian_filtered_images\gaussian_filtered_images',
                                                  x_col = 'id_code',
                                                  y_col =  'stage',
                                                  target_size=(224, 224),
                                                  color_mode='rgb',
                                                  class_mode='categorical',
                                                  batch_size=16)

Found 2929 validated image filenames belonging to 5 classes.
Found 733 validated image filenames belonging to 5 classes.


Model Building

In [37]:
model=Sequential([
  Conv2D(8,(3,3),padding="valid",input_shape=(224,224,3),activation='relu'),
  MaxPool2D(pool_size=(2,2)),
  BatchNormalization(),
  
  Conv2D(16,(3,3),padding="valid",activation='relu'),
  MaxPool2D(pool_size=(2,2)),
  BatchNormalization(),
  
  Conv2D(32,(3,3),padding="valid",activation='relu'),
  MaxPool2D(pool_size=(2,2)),
  BatchNormalization(),
  
  Conv2D(64,(3,3),padding="valid",activation='relu'),
  MaxPool2D(pool_size=(2,2)),
  BatchNormalization(),
  
  Flatten(),
  Dense(32,activation='relu'),
  Dropout(0.25),
  Dense(64,activation='relu'),
  Dropout(0.25),
  Dense(5,activation='softmax')
])

model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_7 (Conv2D)               │ (None, 222, 222, 8)    │           224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 111, 111, 8)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 111, 111, 8)    │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 109, 109, 16)   │         1,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 54, 54, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 54, 54, 16)     │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 52, 52, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 26, 26, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 26, 26, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 24, 24, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 12, 12, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 9216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 32)             │       294,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 322,389 (1.23 MB)

 Trainable params: 322,149 (1.23 MB)

 Non-trainable params: 240 (960.00 B)

In [38]:
optimizer=Adam(learning_rate=1e-05)
model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['acc'])

history=model.fit(train_generator,
                  validation_data=test_generator,
                  steps_per_epoch=train_generator.n//test_generator.batch_size,
                  validation_steps=test_generator.n//test_generator.batch_size,
                  epochs=50)

Epoch 1/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 37s 164ms/step - acc: 0.3195 - loss: 1.6213 - val_acc: 0.2903 - val_loss: 1.4805
Epoch 2/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 0s 366us/step - acc: 0.5000 - loss: 1.0541 - val_acc: 0.3077 - val_loss: 1.4724
Epoch 3/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 30s 153ms/step - acc: 0.5010 - loss: 1.2560 - val_acc: 0.3375 - val_loss: 1.4523
Epoch 4/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 0s 275us/step - acc: 0.5000 - loss: 1.1698 - val_acc: 0.3077 - val_loss: 1.4187
Epoch 5/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 29s 153ms/step - acc: 0.5407 - loss: 1.1633 - val_acc: 0.6514 - val_loss: 0.9931
Epoch 6/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 0s 273us/step - acc: 0.6875 - loss: 0.8595 - val_acc: 0.5385 - val_loss: 1.1873
Epoch 7/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 28s 145ms/step - acc: 0.5864 - loss: 1.1396 - val_acc: 0.6250 - val_loss: 0.9769
Epoch 8/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 0s 250us/step - acc: 0.5000 - loss: 1.3705 - val_acc: 0.6154 - val_loss: 0.9350
Epoch 9/50
183/183 ━━━━━━━━━━━━━━━━━

In [ ]:
## Evaluation of model performance
score = model.evaluate(test_generator)
print('Test accuracy:', score[1])

46/46 ━━━━━━━━━━━━━━━━━━━━ 3s 55ms/step - acc: 0.6821 - loss: 0.8796
Test accuracy: 0.6766712069511414


Fine-tuning of model

In [42]:
model1=Sequential([
  Conv2D(16,(3,3),padding="valid",input_shape=(224,224,3),activation='relu'),
  MaxPool2D(pool_size=(2,2)),
  BatchNormalization(),
  
  Conv2D(32,(3,3),padding="valid",activation='relu'),
  MaxPool2D(pool_size=(2,2)),
  BatchNormalization(),
  
  Conv2D(64,(3,3),padding="valid",activation='relu'),
  MaxPool2D(pool_size=(2,2)),
  BatchNormalization(),
  
  Conv2D(128,(3,3),padding="valid",activation='relu'),
  MaxPool2D(pool_size=(2,2)),
  BatchNormalization(),
  
  Flatten(),
  Dense(32,activation='relu'),
  Dropout(0.15),
  Dense(64,activation='relu'),
  Dropout(0.15),
  Dense(5,activation='softmax')
])

model1.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_15 (Conv2D)              │ (None, 222, 222, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 111, 111, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_15          │ (None, 111, 111, 16)   │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 109, 109, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 54, 54, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_16          │ (None, 54, 54, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 52, 52, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ (None, 26, 26, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_17          │ (None, 26, 26, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 24, 24, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_18          │ (None, 12, 12, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 18432)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 32)             │       589,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 690,693 (2.63 MB)

 Trainable params: 690,213 (2.63 MB)

 Non-trainable params: 480 (1.88 KB)

In [43]:
optimizer=Adam(learning_rate=1e-05)
model1.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['acc'])

history=model1.fit(train_generator,
                  validation_data=test_generator,
                  steps_per_epoch=train_generator.n//test_generator.batch_size,
                  validation_steps=test_generator.n//test_generator.batch_size,
                  epochs=50)

Epoch 1/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 27s 127ms/step - acc: 0.3510 - loss: 1.5035 - val_acc: 0.2861 - val_loss: 1.5414
Epoch 2/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 0s 257us/step - acc: 0.6250 - loss: 1.0836 - val_acc: 0.5385 - val_loss: 1.4183
Epoch 3/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 23s 120ms/step - acc: 0.5922 - loss: 1.1017 - val_acc: 0.3083 - val_loss: 1.5121
Epoch 4/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 0s 174us/step - acc: 0.5625 - loss: 1.1019 - val_acc: 0.2308 - val_loss: 1.5244
Epoch 5/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 23s 123ms/step - acc: 0.6317 - loss: 1.0468 - val_acc: 0.6625 - val_loss: 0.9437
Epoch 6/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 0s 190us/step - acc: 0.4375 - loss: 1.4144 - val_acc: 0.7692 - val_loss: 0.8665
Epoch 7/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 23s 124ms/step - acc: 0.6371 - loss: 0.9953 - val_acc: 0.6750 - val_loss: 0.9280
Epoch 8/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 0s 229us/step - acc: 0.7500 - loss: 0.9243 - val_acc: 0.8462 - val_loss: 0.6069
Epoch 9/50
183/183 ━━━━━━━━━━━━━━━━━

In [44]:
score = model1.evaluate(test_generator)
print('Test accuracy:', score[1])

46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - acc: 0.7020 - loss: 0.9369
Test accuracy: 0.7189631462097168


In [45]:
model2=Sequential([
  Conv2D(16,(3,3),padding="valid",input_shape=(224,224,3),activation='relu'),
  MaxPool2D(pool_size=(2,2)),
  BatchNormalization(),
  
  Conv2D(32,(3,3),padding="valid",activation='relu'),
  MaxPool2D(pool_size=(2,2)),
  BatchNormalization(),
  
  Conv2D(64,(3,3),padding="valid",activation='relu'),
  MaxPool2D(pool_size=(2,2)),
  BatchNormalization(),
  
  Conv2D(128,(3,3),padding="valid",activation='relu'),
  MaxPool2D(pool_size=(2,2)),
  BatchNormalization(),
  
  Flatten(),
  Dense(16,activation='relu'),
  Dropout(0.15),
  Dense(32,activation='relu'),
  Dropout(0.15),
  Dense(64,activation='relu'),
  Dropout(0.15),
  Dense(5,activation='softmax')
])

model2.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_19 (Conv2D)              │ (None, 222, 222, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 111, 111, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_19          │ (None, 111, 111, 16)   │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 109, 109, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_20 (MaxPooling2D) │ (None, 54, 54, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_20          │ (None, 54, 54, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_21 (Conv2D)              │ (None, 52, 52, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_21 (MaxPooling2D) │ (None, 26, 26, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_21          │ (None, 26, 26, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_22 (Conv2D)              │ (None, 24, 24, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_22 (MaxPooling2D) │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_22          │ (None, 12, 12, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 18432)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 16)             │       294,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 32)             │           544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 396,309 (1.51 MB)

 Trainable params: 395,829 (1.51 MB)

 Non-trainable params: 480 (1.88 KB)

In [46]:
optimizer=Adam(learning_rate=1e-05)
model2.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['acc'])

history=model2.fit(train_generator,
                  validation_data=test_generator,
                  steps_per_epoch=train_generator.n//test_generator.batch_size,
                  validation_steps=test_generator.n//test_generator.batch_size,
                  epochs=50)

Epoch 1/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 26s 122ms/step - acc: 0.3031 - loss: 1.4656 - val_acc: 0.4806 - val_loss: 1.5294
Epoch 2/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 0s 263us/step - acc: 0.5625 - loss: 1.0047 - val_acc: 0.3846 - val_loss: 1.5508
Epoch 3/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 23s 121ms/step - acc: 0.4827 - loss: 1.2329 - val_acc: 0.5583 - val_loss: 1.4694
Epoch 4/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 0s 198us/step - acc: 0.5625 - loss: 1.2366 - val_acc: 0.5385 - val_loss: 1.4945
Epoch 5/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 23s 123ms/step - acc: 0.5346 - loss: 1.1447 - val_acc: 0.5875 - val_loss: 1.2351
Epoch 6/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 0s 268us/step - acc: 0.2500 - loss: 1.3327 - val_acc: 0.6923 - val_loss: 1.2594
Epoch 7/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 24s 126ms/step - acc: 0.5532 - loss: 1.0765 - val_acc: 0.6375 - val_loss: 0.9943
Epoch 8/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 0s 222us/step - acc: 0.4375 - loss: 1.2105 - val_acc: 0.4615 - val_loss: 1.4513
Epoch 9/50
183/183 ━━━━━━━━━━━━━━━━━

In [47]:
score = model2.evaluate(test_generator)
print('Test accuracy:', score[1])

46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - acc: 0.6990 - loss: 0.8458
Test accuracy: 0.7107776403427124
